<a href="https://colab.research.google.com/github/pullz6/Research_inspired_Self-Explaining-Neural-Networks-for-Business-Process-Monitoring/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing libraries

In [1]:
!pip install mlflow -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 129.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 700.2/700.2 kB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.9 MB/s eta 0:00:00


In [2]:
!pip install pyngrok -q

In [3]:
#Importing libraries
import mlflow
from mlflow.data.pandas_dataset import PandasDataset
import subprocess
from pyngrok import ngrok, conf
import getpass


import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import datetime
from sklearn.preprocessing import MinMaxScaler

In [4]:
get_ipython().system_raw("mlflow ui --port 5000 &")

In [5]:
ngrok.kill()

In [6]:
NGROK_AUTH_TOKEN = "your code"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

In [7]:
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

MLflow Tracking UI: https://3170-34-125-107-118.ngrok-free.app


In [8]:
df = pd.read_csv("Helpdesk.csv")

In [9]:
df.head()

,concept:name,lifecycle:transition,org:resource,time:timestamp,Activity,Resource,case:concept:name,case:variant,case:variant-index,case:creator
0,Assign seriousness,complete,Value 2,2010-01-13 08:40:25+02:00,Assign seriousness,Value 2,Case3608,Variant 33,33,Fluxicon Disco
1,Take in charge ticket,complete,Value 2,2010-01-29 08:52:27+02:00,Take in charge ticket,Value 2,Case3608,Variant 33,33,Fluxicon Disco
2,Resolve ticket,complete,Value 2,2010-01-29 08:52:34+02:00,Resolve ticket,Value 2,Case3608,Variant 33,33,Fluxicon Disco
3,Closed,complete,Value 5,2010-02-13 08:52:48+02:00,Closed,Value 5,Case3608,Variant 33,33,Fluxicon Disco
4,Closed,complete,Value 5,2010-02-13 08:52:48+02:00,Closed,Value 5,Case3608,Variant 33,33,Fluxicon Disco


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21348 entries, 0 to 21347
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   concept:name          21348 non-null  object
 1   lifecycle:transition  21348 non-null  object
 2   org:resource          21348 non-null  object
 3   time:timestamp        21348 non-null  object
 4   Activity              21348 non-null  object
 5   Resource              21348 non-null  object
 6   case:concept:name     21348 non-null  object
 7   case:variant          21348 non-null  object
 8   case:variant-index    21348 non-null  int64 
 9   case:creator          21348 non-null  object
dtypes: int64(1), object(9)
memory usage: 1.6+ MB


In [11]:
df.drop_duplicates(inplace=True)

In [12]:
df['case:variant-index'] = df['case:variant-index'].apply(lambda x: int(x))

In [13]:
df['Resource'] = df['Resource'].str.strip().str[-1]

In [14]:
df['Resource'] = df['Resource'].apply(lambda x: int(x))

In [15]:
df['time:timestamp'] = pd.to_datetime(df['time:timestamp'], utc=True)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21221 entries, 0 to 21347
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   concept:name          21221 non-null  object             
 1   lifecycle:transition  21221 non-null  object             
 2   org:resource          21221 non-null  object             
 3   time:timestamp        21221 non-null  datetime64[ns, UTC]
 4   Activity              21221 non-null  object             
 5   Resource              21221 non-null  int64              
 6   case:concept:name     21221 non-null  object             
 7   case:variant          21221 non-null  object             
 8   case:variant-index    21221 non-null  int64              
 9   case:creator          21221 non-null  object             
dtypes: datetime64[ns, UTC](1), int64(2), object(7)
memory usage: 1.8+ MB


In [17]:
df['new_date'] = df['time:timestamp'].dt.date
df['new_date'] = pd.to_datetime(df['new_date'], utc=True)
df['new_time'] = pd.to_datetime(
    df['time:timestamp'].dt.time.astype(str),  # Convert time to string
    format='%H:%M:%S'  # Parse as pure time
).dt.floor('S')

<ipython-input-17-16d55716170b>:6: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  ).dt.floor('S')


In [18]:
min_timestamps = df.groupby('case:concept:name')['time:timestamp'].transform('min')

In [19]:
df['time_lapsed'] = df['time:timestamp'] - min_timestamps

In [20]:
df['time_lapsed'] = pd.to_timedelta(df['time_lapsed'])

In [21]:
df['time_lapsed'] = df['time_lapsed'].dt.total_seconds()

In [22]:
max_len = len(df['concept:name'].unique())

In [23]:
#Checking if we can concept:name and activity is the same as well as the org:resource and the resource, so that we can delete this feild.
df['Checker_1'] = (df['concept:name'] == df['Activity'])
df['Checker_2'] = (df['org:resource'] == df['Resource'])

In [24]:
#Let's see if it all the same, if True.
df['Checker_2'].unique()

array([False])

In [25]:
#Lets drop it since it is the same
df.drop(['concept:name','org:resource','Checker_1','Checker_2','case:creator','lifecycle:transition','case:variant'],axis=1, inplace=True)

In [26]:
#Lets create a stage number for each activity stage of the help desk, this will help us easily identify where the case is in terms of the process

#This is also to immulate the label encoder
#First create a dataframe with stage numbers and the stage names
stage = pd.DataFrame()
stage['Stage_no'] = [1,2,3,4,5,6,7,7.1,7.2,8,9,9.1,9.2,10]
stage['Activity'] = ['Insert ticket','Assign seriousness','Take in charge ticket','Wait','Require upgrade','Schedule intervention','Resolve ticket','Create SW anomaly','Resolve SW anomaly','VERIFIED','RESOLVED','INVALID', 'DUPLICATE','Closed']

In [27]:
df=pd.merge(df,stage,on='Activity', how='left')

In [28]:
df.head()

,time:timestamp,Activity,Resource,case:concept:name,case:variant-index,new_date,new_time,time_lapsed,Stage_no
0,2010-01-13 06:40:25+00:00,Assign seriousness,2,Case3608,33,2010-01-13 00:00:00+00:00,1900-01-01 06:40:25,0.0,2.0
1,2010-01-29 06:52:27+00:00,Take in charge ticket,2,Case3608,33,2010-01-29 00:00:00+00:00,1900-01-01 06:52:27,1383122.0,3.0
2,2010-01-29 06:52:34+00:00,Resolve ticket,2,Case3608,33,2010-01-29 00:00:00+00:00,1900-01-01 06:52:34,1383129.0,7.0
3,2010-02-13 06:52:48+00:00,Closed,5,Case3608,33,2010-02-13 00:00:00+00:00,1900-01-01 06:52:48,2679143.0,10.0
4,2010-01-13 10:26:04+00:00,Assign seriousness,2,Case2748,1,2010-01-13 00:00:00+00:00,1900-01-01 10:26:04,0.0,2.0


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21221 entries, 0 to 21220
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   time:timestamp      21221 non-null  datetime64[ns, UTC]
 1   Activity            21221 non-null  object             
 2   Resource            21221 non-null  int64              
 3   case:concept:name   21221 non-null  object             
 4   case:variant-index  21221 non-null  int64              
 5   new_date            21221 non-null  datetime64[ns, UTC]
 6   new_time            21221 non-null  datetime64[ns]     
 7   time_lapsed         21221 non-null  float64            
 8   Stage_no            21221 non-null  float64            
dtypes: datetime64[ns, UTC](2), datetime64[ns](1), float64(2), int64(2), object(2)
memory usage: 1.5+ MB


In [30]:
df.head()

,time:timestamp,Activity,Resource,case:concept:name,case:variant-index,new_date,new_time,time_lapsed,Stage_no
0,2010-01-13 06:40:25+00:00,Assign seriousness,2,Case3608,33,2010-01-13 00:00:00+00:00,1900-01-01 06:40:25,0.0,2.0
1,2010-01-29 06:52:27+00:00,Take in charge ticket,2,Case3608,33,2010-01-29 00:00:00+00:00,1900-01-01 06:52:27,1383122.0,3.0
2,2010-01-29 06:52:34+00:00,Resolve ticket,2,Case3608,33,2010-01-29 00:00:00+00:00,1900-01-01 06:52:34,1383129.0,7.0
3,2010-02-13 06:52:48+00:00,Closed,5,Case3608,33,2010-02-13 00:00:00+00:00,1900-01-01 06:52:48,2679143.0,10.0
4,2010-01-13 10:26:04+00:00,Assign seriousness,2,Case2748,1,2010-01-13 00:00:00+00:00,1900-01-01 10:26:04,0.0,2.0


One-Hot Encoding

In [31]:
from sklearn.preprocessing import OneHotEncoder

In [32]:
one_hot_enc = OneHotEncoder(sparse_output=False)

In [33]:
arr = one_hot_enc.fit_transform(df[['Stage_no']])

In [34]:
feature_names = one_hot_enc.get_feature_names_out(['Stage_no'])

In [35]:
df_encoded = pd.DataFrame(arr, columns=feature_names)

In [36]:
df.drop('Stage_no', axis=1, inplace=True)

In [37]:
df = pd.concat([df, df_encoded], axis=1)

In [38]:
df.head()

,time:timestamp,Activity,Resource,case:concept:name,case:variant-index,new_date,new_time,time_lapsed,Stage_no_1.0,Stage_no_2.0,...,Stage_no_5.0,Stage_no_6.0,Stage_no_7.0,Stage_no_7.1,Stage_no_7.2,Stage_no_8.0,Stage_no_9.0,Stage_no_9.1,Stage_no_9.2,Stage_no_10.0
0,2010-01-13 06:40:25+00:00,Assign seriousness,2,Case3608,33,2010-01-13 00:00:00+00:00,1900-01-01 06:40:25,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2010-01-29 06:52:27+00:00,Take in charge ticket,2,Case3608,33,2010-01-29 00:00:00+00:00,1900-01-01 06:52:27,1383122.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2010-01-29 06:52:34+00:00,Resolve ticket,2,Case3608,33,2010-01-29 00:00:00+00:00,1900-01-01 06:52:34,1383129.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2010-02-13 06:52:48+00:00,Closed,5,Case3608,33,2010-02-13 00:00:00+00:00,1900-01-01 06:52:48,2679143.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,2010-01-13 10:26:04+00:00,Assign seriousness,2,Case2748,1,2010-01-13 00:00:00+00:00,1900-01-01 10:26:04,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
df.columns

Index(['time:timestamp', 'Activity', 'Resource', 'case:concept:name',
       'case:variant-index', 'new_date', 'new_time', 'time_lapsed',
       'Stage_no_1.0', 'Stage_no_2.0', 'Stage_no_3.0', 'Stage_no_4.0',
       'Stage_no_5.0', 'Stage_no_6.0', 'Stage_no_7.0', 'Stage_no_7.1',
       'Stage_no_7.2', 'Stage_no_8.0', 'Stage_no_9.0', 'Stage_no_9.1',
       'Stage_no_9.2', 'Stage_no_10.0'],
      dtype='object')

Splitting Data

In [40]:
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [52]:
X_sequences = []
y_act = []
y_time = []

In [53]:
case_groups = df.groupby('case:concept:name')
max_len = 14  # As per your model's expected timesteps
num_features = 16  # Must match your feature count

In [54]:
features_col = ['Stage_no_1.0', 'Stage_no_2.0', 'Stage_no_3.0', 'Stage_no_4.0',
       'Stage_no_5.0', 'Stage_no_6.0', 'Stage_no_7.0', 'Stage_no_7.1',
       'Stage_no_7.2', 'Stage_no_8.0', 'Stage_no_9.0', 'Stage_no_9.1',
       'Stage_no_9.2', 'Stage_no_10.0','Resource','case:variant-index']

In [55]:
sequence = ['Stage_no_1.0', 'Stage_no_2.0', 'Stage_no_3.0', 'Stage_no_4.0',
       'Stage_no_5.0', 'Stage_no_6.0', 'Stage_no_7.0', 'Stage_no_7.1',
       'Stage_no_7.2', 'Stage_no_8.0', 'Stage_no_9.0', 'Stage_no_9.1',
       'Stage_no_9.2', 'Stage_no_10.0']

In [56]:
group_sizes = [len(group) for _, group in df.groupby('case:concept:name')]
print("Group sizes:", sum(group_sizes))

Group sizes: 21221


In [57]:
len(group_sizes)

4580

In [51]:
from sklearn.preprocessing import MinMaxScaler

df['time_lapsed'] = MinMaxScaler().fit_transform(np.array(df['time_lapsed']).reshape(-1,1))

In [58]:
for _, group in case_groups:
    #Lets sort of the values
    group = group.sort_values('time:timestamp')

    #Lets do normalization
    group['time_lapsed'] = group['time_lapsed'].astype('float32')

    # Process each row individually
    for _, row in group.iterrows():
        # Get features for this single row (shape: [16] for 16 features)
        features = row[features_col].values

        # Reshape to (1, 16) since pad_sequences expects sequences
        features = features.reshape(1, -1)

        # Pad to (14, 16)
        padded = pad_sequences(
            [features],  # Note the list wrapping
            maxlen=max_len,  # 14
            padding='post',
            truncating='post',
            dtype='float32'
        )[0]  # Take the first element to remove extra dimension
        X_sequences.append(padded)

    #Creating the data for y_act
    temp_y_act = group[sequence].to_numpy()
    temp_y_act = np.vstack([
      temp_y_act[1:,:],  # All rows after first
      np.zeros((1, temp_y_act.shape[1]))  # Add zero row at end
    ])
    y_act.extend(temp_y_act)


    #Creating the data for y_time
    temp_y_time = group['time_lapsed'].to_numpy()
    temp_y_time = np.concatenate([
        temp_y_time[1:],  # Elements after first
        [0]  # Append zero
    ])
    y_time.extend(temp_y_time)

In [59]:
y_act = np.array(y_act)
X_sequences = np.array(X_sequences)
y_time = np.array(y_time)

In [60]:
print('Size of X:'+ str(X_sequences.shape))
print('Size of y_act:'+ str((y_act).shape))
print('Size of y_time:'+ str((y_time).shape))

Size of X:(21221, 14, 16)
Size of y_act:(21221, 14)
Size of y_time:(21221,)


In [61]:
print('Sample of X:'+ str(X_sequences[0]))
print('Sample of y_act:'+ str(y_act[0]))
print('Sample of y_time:'+ str(y_time[0]))

Sample of X:[[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1. 12.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
Sample of y_act:[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.

In [ ]:
#Logging the dataset into MLFlow
dataset_source_url = 'https://github.com/pullz6/Research_inspired_Self-Explaining-Neural-Networks-for-Business-Process-Monitoring/blob/main/Data/Helpdesk.csv'
dataset = mlflow.data.from_pandas(
    df, source=dataset_source_url, name="HelpDesk", targets="Activity"
)

Model building

In [50]:
!pip install keras

In [51]:
#Creating the model architecture

In [52]:
#Importing the required libriares since the LSTM model will be done with Pytorch
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.layers import LSTM, GRU, SimpleRNN
from keras.layers import Input
from keras.utils import get_file
from keras.optimizers import Nadam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers import BatchNormalization
from tensorflow.keras.layers import GlobalAveragePooling1D

In [53]:
main_input = Input(shape=(max_len, num_features), name='main_input')

In [54]:
main_input

<KerasTensor shape=(None, 14, 16), dtype=float32, sparse=False, name=main_input>

In [55]:
#Create the training split

In [56]:
l1 = LSTM(100, implementation=2, kernel_initializer='glorot_uniform', return_sequences=True, dropout=0.2)(main_input) # the shared layer

b1 = BatchNormalization()(l1)

l2_1 = LSTM(100, implementation=2, kernel_initializer='glorot_uniform', return_sequences=False, dropout=0.2)(b1) # the layer specialized in activity prediction

b2_1 = BatchNormalization()(l2_1)

l2_2 = LSTM(100, implementation=2, kernel_initializer='glorot_uniform', return_sequences=False, dropout=0.2)(b1) # the layer specialized in time prediction

b2_2 = BatchNormalization()(l2_2)

act_output = Dense(14, activation='softmax', kernel_initializer='glorot_uniform', name='act_output')(b2_1)

time_output = Dense(1, kernel_initializer='glorot_uniform', name='time_output')(b2_2)

#time_output = GlobalAveragePooling1D()(b2_2)  # Reduces (14,64) to (64,)
#time_output = Dense(1, kernel_initializer='glorot_uniform', name='time_output')#(time_output)

In [57]:
learning_rate = 0.002
beta_1 = 0.9
beta_2 = 0.999
epsilon = 1e-08
weight_decay = 0.004
clipvalue = 3
model = Model(inputs=[main_input], outputs=[act_output, time_output])

opt = Nadam(learning_rate=learning_rate, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon, weight_decay=weight_decay, clipvalue=clipvalue)

model.compile(loss={'act_output':'categorical_crossentropy', 'time_output':'mae'}, optimizer=opt)

early_stopping = EarlyStopping(monitor='val_loss', patience=42)

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

Model Training

In [58]:
model.fit(X_sequences, {'act_output':y_act, 'time_output':y_time}, validation_split=0.2, verbose=2, callbacks=[early_stopping, lr_reducer], batch_size=max_len, epochs=100)

Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['main_input']
Received: inputs=Tensor(shape=(None, 14, 16))
  warnings.warn(msg)


1213/1213 - 50s - 41ms/step - act_output_loss: 0.9762 - loss: 1166844.6250 - time_output_loss: 1166873.3750 - val_act_output_loss: 1.3709 - val_loss: 1167869.8750 - val_time_output_loss: 1170432.3750 - learning_rate: 0.0020
Epoch 2/100
1213/1213 - 40s - 33ms/step - act_output_loss: 1.1815 - loss: 1165256.8750 - time_output_loss: 1165373.3750 - val_act_output_loss: 1.0505 - val_loss: 1164692.8750 - val_time_output_loss: 1167251.7500 - learning_rate: 0.0020
Epoch 3/100
1213/1213 - 41s - 34ms/step - act_output_loss: 1.3223 - loss: 1162979.3750 - time_output_loss: 1162710.3750 - val_act_output_loss: 1.2125 - val_loss: 1162371.8750 - val_time_output_loss: 1164930.0000 - learning_rate: 0.0020
Epoch 4/100
1213/1213 - 39s - 32ms/step - act_output_loss: 1.4049 - loss: 1160194.8750 - time_output_loss: 1160434.1250 - val_act_output_loss: 1.1497 - val_loss: 1158761.2500 - val_time_output_loss: 1161316.8750 - learning_rate: 0.0020
Epoch 5/100
1213/1213 - 41s - 34ms/step - act_output_loss: 1.5584 - 

In [64]:
with mlflow.start_run(run_name="Activity Prediction Model"):
  model_info = mlflow.sklearn.log_model(
          sk_model=model,
          artifact_path="Prediction Model",
          input_example=X_sequences,
          registered_model_name="Prediction-Model",
      )

664/664 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step
664/664 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step


MlflowException: When an mlflow-artifacts URI was supplied, the tracking URI must be a valid http or https URI, but it was currently set to file:///content/mlruns. Perhaps you forgot to set the tracking URI to the running MLflow server. To set the tracking URI, use either of the following methods:
1. Set the MLFLOW_TRACKING_URI environment variable to the desired tracking URI. `export MLFLOW_TRACKING_URI=http://localhost:5000`
2. Set the tracking URI programmatically by calling `mlflow.set_tracking_uri`. `mlflow.set_tracking_uri('http://localhost:5000')`